<a href="https://colab.research.google.com/github/taaha3244/Pytorch-Lightning/blob/main/Chapter1_XOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing pytorch lightning 
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.0 MB/s eta 0:00:00


In [2]:
#import all the libraries
import pytorch_lightning as pl
import torch
from torch import nn, optim
from torch.autograd import Variable
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
print("torch version:",torch.__version__)
print("pytorch ligthening version:",pl.__version__)

torch version: 2.0.1+cu118
pytorch ligthening version: 2.0.2


In [3]:
#Building a simple DataLoader for XOR data
xor_inputs=[torch.Tensor([0,0]),
            torch.Tensor([0,1]),
            torch.Tensor([1,0]),
            torch.Tensor([1,1])]
xor_inputs

[tensor([0., 0.]), tensor([0., 1.]), tensor([1., 0.]), tensor([1., 1.])]

In [4]:
xor_targets=[torch.Tensor([0]),
             torch.Tensor([1]),
             torch.Tensor([1]),
             torch.Tensor([0])]
xor_targets

[tensor([0.]), tensor([1.]), tensor([1.]), tensor([0.])]

In [5]:
xor_data = list(zip(xor_inputs, xor_targets))
train_loader = DataLoader(xor_data, batch_size=1)


In [6]:
feature,label=next(iter(train_loader))
feature,label

(tensor([[0., 0.]]), tensor([[0.]]))

In [7]:
class XorModel(pl.LightningModule):
  def __init__(self):
    super(XorModel,self).__init__()
    self.input_layer=nn.Linear(2,4)
    self.output_layer=nn.Linear(4,1)
    self.sigmoid=nn.Sigmoid()
    self.loss=nn.MSELoss()
  def forward(self,input):
    x=self.input_layer(input)
    x=self.sigmoid(x)
    output=self.output_layer(x)
    return output
  def configure_optimizers(self):
    params = self.parameters()
    optimizer = optim.Adam(params=params, lr = 0.01)
    return optimizer
  def training_step(self, batch, batch_idx):
    xor_inputs, xor_targets = batch
    #print("XOR INPUT:", xor_input.shape)
    #print("XOR TARGET:", xor_target.shape)
    outputs = self(xor_inputs) 
    #print("XOR OUTPUT:", outputs.shape)
    loss = self.loss(outputs, xor_targets)
    return loss 

In [8]:
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS
checkpoint_callback = ModelCheckpoint()
model = XorModel()
trainer = pl.Trainer(max_epochs=100, callbacks=[checkpoint_callback])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(model, train_dataloaders=train_loader)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type    | Params
-----------------------------------------
0 | input_layer  | Linear  | 12    
1 | output_layer | Linear  | 5     
2 | sigmoid      | Sigmoid | 0     
3 | loss         | MSELoss | 0     
-----------------------------------------
17        Trainable params
0         Non-trainable params
17        Total params
0.000     Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [10]:
#Lightning Logs has data of various model instances being trained ,saved in seperate folders
ls lightning_logs/

version_0/


In [11]:
#Checking the best model
print(checkpoint_callback.best_model_path)

/content/lightning_logs/version_0/checkpoints/epoch=99-step=400.ckpt


In [12]:
#Loading the best model
train_model = model.load_from_checkpoint(checkpoint_callback.best_model_path)

In [14]:
test = torch.utils.data.DataLoader(xor_inputs, batch_size=1)
for val in xor_inputs:
  y= train_model(val)
  print([int(val[0]),int(val[1])],int(y.round()))

[0, 0] 0
[0, 1] 1
[1, 0] 1
[1, 1] 1
